In [1]:
#Initialize dependencies
import tensorflow as tf
import os
BASE_FOLDER = '/Users/oblivisheee/Documents/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
%store -r
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.data_preprocess import DataGenerator


/Users/oblivisheee/miniconda3/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:

dataset_dirs = {
    'train_russian': os.path.join(BASE_FOLDER, 'dataset/train/russian'),
    'train_english': os.path.join(BASE_FOLDER, 'dataset/train/english'),
    'valid_russian': os.path.join(BASE_FOLDER, 'dataset/valid/russian'),
    'valid_english': os.path.join(BASE_FOLDER, 'dataset/valid/english')
}

data_gen = DataGenerator(train_russian_dir=dataset_dirs['train_russian'], 
                         train_english_dir=dataset_dirs['train_english'], 
                         valid_russian_dir=dataset_dirs['valid_russian'], 
                         valid_english_dir=dataset_dirs['valid_english'])
(train_russian_data, train_english_data, valid_russian_data, valid_english_data) = data_gen.generate()

dataset_train = {
    'input': tf.data.Dataset.from_tensor_slices(train_english_data),
    'target': tf.data.Dataset.from_tensor_slices(train_russian_data),
}
dataset_valid = {
    'input': tf.data.Dataset.from_tensor_slices(valid_english_data),
    'target': tf.data.Dataset.from_tensor_slices(valid_russian_data)
}



# Config training

In [ ]:
# Configure the Transformer layer

embedding_layer = tf.keras.layers.Embedding(8500, 512)
regularized_layer = RegularizedDenseLayer(512)

model = Transformer(num_layers=6, d_model=512, num_heads=8, dff=2048,
                          input_vocab_size=8500, target_vocab_size=8500, pe_input=10000,
                          pe_target=10000, rate=0.1, embedding=embedding_layer, regularized_layer=regularized_layer)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='auto')
optimizer = tf.keras.optimizers.legacy.Adam(name='Adam',learning_rate=0.001,
                                     epsilon=1e-8, amsgrad=True,
                                     beta_1=0.9, beta_2=0.999)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'), 
           tf.keras.metrics.MeanSquaredError(name='mean_squared_error'), 
           tf.keras.metrics.Precision(thresholds=0.5, name='precision'),]


model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)

# Start training

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
model.fit_model(input_dataset=dataset_train['input'], 
                target_dataset=dataset_train['target'], 
                valid_input_dataset=dataset_valid['input'], 
                valid_target_dataset=dataset_valid['target'], 
                epochs=10,
                session_name='novelsdreamer-test',
                batch_size=64, gradient_accumulation_steps=5,
                shuffle=True,
                save_model_each_epoch=True,
                model_name='novelsdreamer-ru-t4m')